# Training and saving keras classifier

## Importing libraries

In [201]:
from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, Flatten
from keras.layers.convolutional import Conv2D
from keras.datasets import fashion_mnist
from keras.losses import categorical_crossentropy
import numpy as np
from random import shuffle
import pickle, os, cv2
from time import *

## Creating a neural network

In [202]:
model = Sequential()
model.add(Conv2D(1, strides=(1, 1), activation='relu', kernel_size=(3,3), input_shape=(20,20,1), data_format="channels_last"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dense(500, activation='relu'))
model.add(Conv2D(150, (5, 5), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(800, activation='relu'))
model.add(Dense(18, activation='softmax'))

## Compiling

In [203]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Importing data

In [217]:
dataset = np.loadtxt("data.csv", delimiter=",").astype(int)
L = pickle.load(open("categories.sav", "rb"))
X, Y = dataset[:,1:], dataset[:,0]
n, m = dataset.shape
labels = []
images = []
for i in range(n):
    l = [0]*18
    l[Y[i]] = 1
    labels.append(l)
    images.append(255-X[i].reshape(20,20))
    
new_path = "./sheets/Rebelo Dataset/database1/database/syn/"
repositories = ["altoClef", "beams", "flat", "naturals", "notesFlags", "notesOpen", "rests1", "rests2", "sharps", "time", "trebleClef"]
corresponding_labels = [0,3,6,7,8,9,10,11,12,14,15,17]
new_images = []
new_labels = []
for repo in repositories:
    fresh_path = new_path + repo + "/"
    for filename in os.listdir(fresh_path):
        file = fresh_path + filename
        img = cv2.imread(file, 0)
        img = cv2.resize(img, (20,20))
        new_images.append(255-img)
        l = [0]*18
        l[corresponding_labels[repositories.index(repo)]] = 1
        new_labels.append(l)
        
images.extend(new_images)
labels.extend(new_labels)
n = len(images)

combination = list(zip(images,labels))
shuffle(combination)

images, labels = zip(*combination)

images = np.array(images)
images = images.reshape(images.shape[0],20,20,1)
labels = np.array(labels)
#labels = labels.reshape((labels.shape[1], labels.shape[0]))
#(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()



In [218]:
print(labels.shape)
print(images.shape)

(9414, 18)
(9414, 20, 20, 1)


## Splitting training and testing sets

In [219]:
train_number = n * 75 // 100
images_train, images_test, labels_train, labels_test = images[:train_number,:,:,:], images[train_number:,:,:,:], labels[:train_number], labels[train_number:]

In [220]:
print(labels_train.shape)

(7060, 18)


## Training

In [221]:
model.fit(images_train, labels_train)

Epoch 1/1
7060/7060 [==============================] - 44s 6ms/step - loss: 0.9196 - acc: 0.7429


## Testing

In [222]:
loss_and_metrics = model.evaluate(images_test, labels_test)

2354/2354 [==============================] - 4s 2ms/step


In [223]:
print(loss_and_metrics)

[0.5149935523622504, 0.8453695836873407]


In [224]:
modelname = "symbols_classifier.sav"
pickle.dump(model, open(modelname, 'wb'))